In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score,train_test_split, KFold, cross_val_predict
from sklearn.metrics import mean_absolute_error, mean_squared_error,r2_score,roc_curve,auc,precision_recall_curve, accuracy_score, \
recall_score, precision_score, confusion_matrix
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, ParameterGrid, StratifiedKFold, RandomizedSearchCV
from sklearn.ensemble import VotingRegressor, VotingClassifier, StackingRegressor, StackingClassifier, GradientBoostingRegressor,GradientBoostingClassifier, BaggingRegressor,BaggingClassifier,RandomForestRegressor,RandomForestClassifier,AdaBoostRegressor,AdaBoostClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression, LassoCV, RidgeCV, ElasticNetCV
from sklearn.neighbors import KNeighborsRegressor
import itertools as it
import time as time
import xgboost as xgb
from pyearth import Earth

In [2]:
data = pd.read_csv('OnlineNewsPopularity/OnlineNewsPopularity.csv')
data.columns = [x.strip() for x in data.columns.tolist()]
X = data.drop(columns = ['weekday_is_monday',
 'weekday_is_saturday',
 'LDA_00',
 'n_unique_tokens',
 'n_non_stop_words',
 'self_reference_avg_sharess',
 'rate_positive_words',
 'LDA_04',
 'LDA_02',
 'kw_max_max',
 'kw_avg_min',
 'kw_avg_avg',
 'rate_negative_words',
 'min_negative_polarity', 'url', 'timedelta', 'shares'])
y = data['shares']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 45)

# We will start tuning parameters one by one to look for a general range to look for, for when we tune all the parameters at the same time

### Number of trees
We will use the same number of trees - 100 for all future fitting, until the final model, because we will be using the early_stopping_rounds of the XGBRegressor to determine the right number of trees.

### Depth of the model

In [ ]:
# get a list of models to evaluate
def get_models():
    models = dict()
    # explore depths from 1 to 10
    for i in range(1,21):
        # define ensemble model
        models[str(i)] = xgb.XGBRegressor(random_state=1,max_depth=i, objective = 'reg:absoluteerror', eval_metric = mean_absolute_error)
    return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    # define the evaluation procedure
    cv = KFold(n_splits=10, shuffle=True, random_state=1)
    # evaluate the model and collect the results
    scores = -cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    return scores

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    # evaluate the model
    scores = evaluate_model(model, X_train, np.log(y_train))
    scores = np.exp(scores)
    # store the results
    results.append(scores)
    names.append(name)
    # summarize the performance along the way
    print('>%s %.3f (%.3f)' % (name, np.mean(scores), np.std(scores)))
plt.boxplot(results, labels=names, showmeans=True)
plt.ylabel('Cross validation error',fontsize=15)
plt.xlabel('Depth of each tree',fontsize=15)

>1 1.858 (0.018)
>2 1.852 (0.021)
>3 1.854 (0.020)
>4 1.857 (0.019)
>5 1.866 (0.020)
>6 1.874 (0.020)
>7 1.891 (0.018)
>8 1.916 (0.020)
>9 1.946 (0.019)
>10 1.983 (0.023)
>11 2.049 (0.051)
>12 2.079 (0.026)
>13 2.257 (0.087)
>14 2.460 (0.129)
>15 2.753 (0.121)


### Learning rate

In [ ]:
def get_models():
    models = dict()
    # explore learning rates from 0.1 to 2 in 0.1 increments
    for i in [0.01,0.05,0.1,0.2,0.3,0.4,0.5,0.6,0.8,1.0]:
        key = '%.4f' % i
        models[key] = xgb.XGBRegressor(learning_rate=i,random_state=1, objective = 'reg:absoluteerror', eval_metric = mean_absolute_error)
    return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    # define the evaluation procedure
    cv = KFold(n_splits=10, shuffle=True, random_state=1)
    # evaluate the model and collect the results
    scores = -cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    return scores

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    # evaluate the model
    scores = evaluate_model(model, X_train, np.log(y_train))
    scores = np.exp(scores)
    # store the results
    results.append(scores)
    names.append(name)
    # summarize the performance along the way
    print('>%s %.1f (%.1f)' % (name, np.mean(scores), np.std(scores)))
# plot model performance for comparison
plt.figure(figsize=(7, 7))
plt.boxplot(results, labels=names, showmeans=True)
plt.ylabel('Cross validation error',fontsize=15)
plt.xlabel('Learning rate',fontsize=15)

### Regularization

In [ ]:
def get_models():
    models = dict()
    # explore learning rates from 0.1 to 2 in 0.1 increments
    for i in [0,0.5,1.0,1.5,2,10,100]:
        key = '%.4f' % i
        models[key] = xgb.XGBRegressor(reg_lambda=i,random_state=1, objective = 'reg:absoluteerror', eval_metric = mean_absolute_error)
    return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    # define the evaluation procedure
    cv = KFold(n_splits=10, shuffle=True, random_state=1)
    # evaluate the model and collect the results
    scores = -cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    return scores

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    # evaluate the model
    scores = evaluate_model(model, X_train, np.log(y_train))
    scores = np.exp(scores)
    # store the results
    results.append(scores)
    names.append(name)
    # summarize the performance along the way
    print('>%s %.1f (%.1f)' % (name, np.mean(scores), np.std(scores)))
# plot model performance for comparison
plt.figure(figsize=(7, 7))
plt.boxplot(results, labels=names, showmeans=True)
plt.ylabel('Cross validation error',fontsize=15)
plt.xlabel('reg_lambda',fontsize=15)

In [ ]:
# Fine tune more general range for regularization
def get_models():
    models = dict()
    # explore learning rates from 0.1 to 2 in 0.1 increments
    for i in [5,10,50,100,500,1000, 5000, 10000, 50000]:
        key = '%.4f' % i
        models[key] = xgb.XGBRegressor(reg_lambda=i,random_state=1, objective = 'reg:absoluteerror', eval_metric = mean_absolute_error)
    return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    # define the evaluation procedure
    cv = KFold(n_splits=10, shuffle=True, random_state=1)
    # evaluate the model and collect the results
    scores = -cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    return scores

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    # evaluate the model
    scores = evaluate_model(model, X_train, np.log(y_train))
    scores = np.exp(scores)
    # store the results
    results.append(scores)
    names.append(name)
    # summarize the performance along the way
    print('>%s %.1f (%.1f)' % (name, np.mean(scores), np.std(scores)))
# plot model performance for comparison
plt.figure(figsize=(7, 7))
plt.boxplot(results, labels=names, showmeans=True)
plt.ylabel('Cross validation error',fontsize=15)
plt.xlabel('reg_lambda',fontsize=15)

### Regularization in gamma

In [ ]:
def get_models():
    models = dict()
    # explore learning rates from 0.1 to 2 in 0.1 increments
    for i in [0,10,1e2,1e3,1e4,1e5,1e6,1e7,1e8,1e9]:
        key = '%.4f' % i
        models[key] = xgb.XGBRegressor(gamma=i,random_state=1, objective = 'reg:absoluteerror', eval_metric = mean_absolute_error)
    return models

# evaluate a given model using cross-validation
def evaluate_model(model, X, y):
    # define the evaluation procedure
    cv = KFold(n_splits=10, shuffle=True, random_state=1)
    # evaluate the model and collect the results
    scores = -cross_val_score(model, X, y, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)
    return scores

# get the models to evaluate
models = get_models()
# evaluate the models and store results
results, names = list(), list()
for name, model in models.items():
    # evaluate the model
    scores = evaluate_model(model, X_train, np.log(y_train))
    scores = np.exp(scores)
    # store the results
    results.append(scores)
    names.append(name)
    # summarize the performance along the way
    print('>%s %.1f (%.1f)' % (name, np.mean(scores), np.std(scores)))
# plot model performance for comparison
plt.figure(figsize=(7, 7))
plt.boxplot(results, labels=names, showmeans=True)
plt.ylabel('Cross validation error',fontsize=15)
plt.xlabel('gamma',fontsize=15)

### Tuning full model with everything

In [20]:
#K-fold cross validation to find optimal parameters for XGBoost
start_time = time.time()
param_grid = {'max_depth': [2, 3, 4],
              'learning_rate': [0.05, 0.1, 0.2],
               'reg_lambda':[100, 500, 1000, 2000],
                'n_estimators':[50, 100, 500],
                'gamma': [5, 10, 20],
                'subsample': [0.5, 0.75, 1.0],
                'colsample_bytree': [0.5, 0.75, 1.0]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
optimal_params = RandomizedSearchCV(estimator=xgb.XGBRegressor(random_state=1, objective = 'reg:absoluteerror',
                                                              eval_metric = mean_absolute_error),                                                       
                             param_distributions = param_grid, n_iter = 200,
                             verbose = 1,
                             n_jobs=-1,
                             cv = cv)
optimal_params.fit(X_train,np.log(y_train))
print("Optimal parameter values =", optimal_params.best_params_)
print("Optimal cross validation R-squared = ",optimal_params.best_score_)
print("Time taken = ", round((time.time()-start_time)/60), " minutes")

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Optimal parameter values = {'subsample': 0.75, 'reg_lambda': 100, 'n_estimators': 500, 'max_depth': 4, 'learning_rate': 0.1, 'gamma': 5, 'colsample_bytree': 0.5}
Optimal cross validation R-squared =  0.16055439119071493
Time taken =  27  minutes


In [21]:
#K-fold cross validation to find optimal parameters for XGBoost
# Finer search
start_time = time.time()
param_grid = {'max_depth': [2, 3, 4, 5],
              'learning_rate': [0.02, 0.05, 0.1, 0.2],
              'reg_alpha':[0, 10, 25, 50, 100]
               'reg_lambda':[0, 50, 100, 200, 400],
                'n_estimators':[100, 500, 750, 1500],
                'gamma': [2, 5, 8, 11, 14, 17, 20],
                'subsample': [0.5, 0.75, 1],
                'colsample_bytree': [0.5, 0.75, 1]}

cv = KFold(n_splits=5,shuffle=True,random_state=1)
optimal_params = RandomizedSearchCV(estimator=xgb.XGBRegressor(random_state=1),                                                       
                             param_distributions = param_grid, n_iter = 200,
                             verbose = 1,
                             n_jobs=-1,
                             cv = cv)
optimal_params.fit(X_train,np.log(y_train))
print("Optimal parameter values =", optimal_params.best_params_)
print("Optimal cross validation R-squared = ",optimal_params.best_score_)
print("Time taken = ", round((time.time()-start_time)/60), " minutes")

Fitting 5 folds for each of 200 candidates, totalling 1000 fits
Optimal parameter values = {'subsample': 1, 'reg_lambda': 200, 'n_estimators': 1000, 'max_depth': 4, 'learning_rate': 0.12, 'gamma': 2, 'colsample_bytree': 0.5}
Optimal cross validation R-squared =  0.16259412911125098
Time taken =  75  minutes


In [29]:
model = xgb.XGBRegressor(random_state = 1, max_depth = 4, learning_rate = 0.001,
                        n_estimators = 20000,reg_lambda = 100, gamma = 5, subsample = 0.75, colsample_bytree = 0.5, early_stopping_rounds = 250)
model.fit(X_train, y_train, eval_set = ([(X_test, y_test)]))

[0]	validation_0-rmse:8209.78734
[1]	validation_0-rmse:8208.33871
[2]	validation_0-rmse:8206.79883
[3]	validation_0-rmse:8205.38461
[4]	validation_0-rmse:8203.95035
[5]	validation_0-rmse:8202.45072
[6]	validation_0-rmse:8201.01564
[7]	validation_0-rmse:8199.60977
[8]	validation_0-rmse:8198.19039
[9]	validation_0-rmse:8196.69863
[10]	validation_0-rmse:8195.17227
[11]	validation_0-rmse:8193.66593
[12]	validation_0-rmse:8192.11573
[13]	validation_0-rmse:8190.60183
[14]	validation_0-rmse:8189.08032
[15]	validation_0-rmse:8187.68317
[16]	validation_0-rmse:8186.17156
[17]	validation_0-rmse:8184.58588
[18]	validation_0-rmse:8183.06826
[19]	validation_0-rmse:8181.58964
[20]	validation_0-rmse:8180.12418
[21]	validation_0-rmse:8178.77708
[22]	validation_0-rmse:8177.35842
[23]	validation_0-rmse:8175.87074
[24]	validation_0-rmse:8174.46071
[25]	validation_0-rmse:8173.09480
[26]	validation_0-rmse:8171.63403
[27]	validation_0-rmse:8170.14989
[28]	validation_0-rmse:8168.67760
[29]	validation_0-rmse:8

[238]	validation_0-rmse:7915.40354
[239]	validation_0-rmse:7914.45093
[240]	validation_0-rmse:7913.43865
[241]	validation_0-rmse:7912.40691
[242]	validation_0-rmse:7911.35568
[243]	validation_0-rmse:7910.44477
[244]	validation_0-rmse:7909.41965
[245]	validation_0-rmse:7908.44050
[246]	validation_0-rmse:7907.45094
[247]	validation_0-rmse:7906.42032
[248]	validation_0-rmse:7905.41101
[249]	validation_0-rmse:7904.47565
[250]	validation_0-rmse:7903.47156
[251]	validation_0-rmse:7902.50005
[252]	validation_0-rmse:7901.49372
[253]	validation_0-rmse:7900.45689
[254]	validation_0-rmse:7899.43786
[255]	validation_0-rmse:7898.51968
[256]	validation_0-rmse:7897.54404
[257]	validation_0-rmse:7896.56963
[258]	validation_0-rmse:7895.64504
[259]	validation_0-rmse:7894.67083
[260]	validation_0-rmse:7893.65214
[261]	validation_0-rmse:7892.72078
[262]	validation_0-rmse:7891.70695
[263]	validation_0-rmse:7890.71130
[264]	validation_0-rmse:7889.74542
[265]	validation_0-rmse:7888.82098
[266]	validation_0-r

[473]	validation_0-rmse:7721.62776
[474]	validation_0-rmse:7720.98070
[475]	validation_0-rmse:7720.30096
[476]	validation_0-rmse:7719.61334
[477]	validation_0-rmse:7718.92708
[478]	validation_0-rmse:7718.27325
[479]	validation_0-rmse:7717.59160
[480]	validation_0-rmse:7716.95191
[481]	validation_0-rmse:7716.31749
[482]	validation_0-rmse:7715.63783
[483]	validation_0-rmse:7714.98898
[484]	validation_0-rmse:7714.28986
[485]	validation_0-rmse:7713.61713
[486]	validation_0-rmse:7712.94164
[487]	validation_0-rmse:7712.28796
[488]	validation_0-rmse:7711.65698
[489]	validation_0-rmse:7711.02107
[490]	validation_0-rmse:7710.39957
[491]	validation_0-rmse:7709.77771
[492]	validation_0-rmse:7709.13562
[493]	validation_0-rmse:7708.49236
[494]	validation_0-rmse:7707.83208
[495]	validation_0-rmse:7707.18333
[496]	validation_0-rmse:7706.56054
[497]	validation_0-rmse:7705.90197
[498]	validation_0-rmse:7705.23651
[499]	validation_0-rmse:7704.55649
[500]	validation_0-rmse:7703.93633
[501]	validation_0-r

[708]	validation_0-rmse:7592.65029
[709]	validation_0-rmse:7592.18424
[710]	validation_0-rmse:7591.72492
[711]	validation_0-rmse:7591.28314
[712]	validation_0-rmse:7590.85399
[713]	validation_0-rmse:7590.41304
[714]	validation_0-rmse:7590.00840
[715]	validation_0-rmse:7589.59023
[716]	validation_0-rmse:7589.16327
[717]	validation_0-rmse:7588.73670
[718]	validation_0-rmse:7588.31974
[719]	validation_0-rmse:7587.85281
[720]	validation_0-rmse:7587.41814
[721]	validation_0-rmse:7587.01272
[722]	validation_0-rmse:7586.59549
[723]	validation_0-rmse:7586.13452
[724]	validation_0-rmse:7585.69295
[725]	validation_0-rmse:7585.25986
[726]	validation_0-rmse:7584.88238
[727]	validation_0-rmse:7584.47157
[728]	validation_0-rmse:7584.03948
[729]	validation_0-rmse:7583.60163
[730]	validation_0-rmse:7583.16479
[731]	validation_0-rmse:7582.72765
[732]	validation_0-rmse:7582.30979
[733]	validation_0-rmse:7581.88149
[734]	validation_0-rmse:7581.45375
[735]	validation_0-rmse:7581.06483
[736]	validation_0-r

[943]	validation_0-rmse:7507.17842
[944]	validation_0-rmse:7506.85706
[945]	validation_0-rmse:7506.54582
[946]	validation_0-rmse:7506.22940
[947]	validation_0-rmse:7505.90984
[948]	validation_0-rmse:7505.62504
[949]	validation_0-rmse:7505.33281
[950]	validation_0-rmse:7505.05650
[951]	validation_0-rmse:7504.76200
[952]	validation_0-rmse:7504.49054
[953]	validation_0-rmse:7504.16282
[954]	validation_0-rmse:7503.86978
[955]	validation_0-rmse:7503.56375
[956]	validation_0-rmse:7503.26877
[957]	validation_0-rmse:7502.95898
[958]	validation_0-rmse:7502.66853
[959]	validation_0-rmse:7502.34401
[960]	validation_0-rmse:7502.05342
[961]	validation_0-rmse:7501.75128
[962]	validation_0-rmse:7501.46828
[963]	validation_0-rmse:7501.21481
[964]	validation_0-rmse:7500.90906
[965]	validation_0-rmse:7500.58677
[966]	validation_0-rmse:7500.30514
[967]	validation_0-rmse:7500.03002
[968]	validation_0-rmse:7499.73783
[969]	validation_0-rmse:7499.46679
[970]	validation_0-rmse:7499.19760
[971]	validation_0-r

...

[3681]	validation_0-rmse:7335.52159
[3682]	validation_0-rmse:7335.53002
[3683]	validation_0-rmse:7335.51580
[3684]	validation_0-rmse:7335.51195
[3685]	validation_0-rmse:7335.49048
[3686]	validation_0-rmse:7335.49654
[3687]	validation_0-rmse:7335.50407
[3688]	validation_0-rmse:7335.50339
[3689]	validation_0-rmse:7335.49518
[3690]	validation_0-rmse:7335.49899
[3691]	validation_0-rmse:7335.48060
[3692]	validation_0-rmse:7335.47092
[3693]	validation_0-rmse:7335.45791
[3694]	validation_0-rmse:7335.45276
[3695]	validation_0-rmse:7335.44434
[3696]	validation_0-rmse:7335.43054
[3697]	validation_0-rmse:7335.40398
[3698]	validation_0-rmse:7335.39751
[3699]	validation_0-rmse:7335.40148
[3700]	validation_0-rmse:7335.40941
[3701]	validation_0-rmse:7335.39150
[3702]	validation_0-rmse:7335.40839
[3703]	validation_0-rmse:7335.40754
[3704]	validation_0-rmse:7335.41310
[3705]	validation_0-rmse:7335.40828
[3706]	validation_0-rmse:7335.40517
[3707]	validation_0-rmse:7335.42080
[3708]	validation_0-rmse:733

[3909]	validation_0-rmse:7335.04483
[3910]	validation_0-rmse:7335.04822
[3911]	validation_0-rmse:7335.05096
[3912]	validation_0-rmse:7335.05296
[3913]	validation_0-rmse:7335.05637
[3914]	validation_0-rmse:7335.05628
[3915]	validation_0-rmse:7335.06160
[3916]	validation_0-rmse:7335.05285
[3917]	validation_0-rmse:7335.05793
[3918]	validation_0-rmse:7335.05980
[3919]	validation_0-rmse:7335.05231
[3920]	validation_0-rmse:7335.04636
[3921]	validation_0-rmse:7335.03772
[3922]	validation_0-rmse:7335.03468
[3923]	validation_0-rmse:7335.04804
[3924]	validation_0-rmse:7335.05396
[3925]	validation_0-rmse:7335.04933
[3926]	validation_0-rmse:7335.03554
[3927]	validation_0-rmse:7335.03103
[3928]	validation_0-rmse:7335.02238
[3929]	validation_0-rmse:7335.03072
[3930]	validation_0-rmse:7335.04942
[3931]	validation_0-rmse:7335.05457
[3932]	validation_0-rmse:7335.04620
[3933]	validation_0-rmse:7335.04727
[3934]	validation_0-rmse:7335.03800
[3935]	validation_0-rmse:7335.02187
[3936]	validation_0-rmse:733

[4137]	validation_0-rmse:7334.78830
[4138]	validation_0-rmse:7334.78764
[4139]	validation_0-rmse:7334.79419
[4140]	validation_0-rmse:7334.77573
[4141]	validation_0-rmse:7334.77375
[4142]	validation_0-rmse:7334.76798
[4143]	validation_0-rmse:7334.77013
[4144]	validation_0-rmse:7334.76478
[4145]	validation_0-rmse:7334.76478
[4146]	validation_0-rmse:7334.76312
[4147]	validation_0-rmse:7334.76979
[4148]	validation_0-rmse:7334.75819
[4149]	validation_0-rmse:7334.72747
[4150]	validation_0-rmse:7334.72467
[4151]	validation_0-rmse:7334.72590
[4152]	validation_0-rmse:7334.71990
[4153]	validation_0-rmse:7334.74286
[4154]	validation_0-rmse:7334.73147
[4155]	validation_0-rmse:7334.73519
[4156]	validation_0-rmse:7334.73180
[4157]	validation_0-rmse:7334.73369
[4158]	validation_0-rmse:7334.73684
[4159]	validation_0-rmse:7334.74385
[4160]	validation_0-rmse:7334.74138
[4161]	validation_0-rmse:7334.74255
[4162]	validation_0-rmse:7334.74206
[4163]	validation_0-rmse:7334.72976
[4164]	validation_0-rmse:733

[4365]	validation_0-rmse:7334.75213
[4366]	validation_0-rmse:7334.76185
[4367]	validation_0-rmse:7334.77105
[4368]	validation_0-rmse:7334.77722
[4369]	validation_0-rmse:7334.79069
[4370]	validation_0-rmse:7334.77992
[4371]	validation_0-rmse:7334.78147
[4372]	validation_0-rmse:7334.77467
[4373]	validation_0-rmse:7334.77090
[4374]	validation_0-rmse:7334.77131
[4375]	validation_0-rmse:7334.75201
[4376]	validation_0-rmse:7334.75474
[4377]	validation_0-rmse:7334.75260
[4378]	validation_0-rmse:7334.76195
[4379]	validation_0-rmse:7334.75190
[4380]	validation_0-rmse:7334.75310
[4381]	validation_0-rmse:7334.75982
[4382]	validation_0-rmse:7334.75848
[4383]	validation_0-rmse:7334.76132
[4384]	validation_0-rmse:7334.76440
[4385]	validation_0-rmse:7334.77527
[4386]	validation_0-rmse:7334.76608
[4387]	validation_0-rmse:7334.75985
[4388]	validation_0-rmse:7334.76151
[4389]	validation_0-rmse:7334.74987
[4390]	validation_0-rmse:7334.77068
[4391]	validation_0-rmse:7334.77097
[4392]	validation_0-rmse:733

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, early_stopping_rounds=250,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=5, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.001, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=20000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [30]:
print('XGBoost MAE is {}'.format(mean_absolute_error(y_test, model.predict(X_test))))

XGBoost MAE is 2819.469302557143


In [27]:
model = xgb.XGBRegressor(random_state = 1, max_depth = 4, learning_rate = 0.001,
                        n_estimators = 20000,reg_lambda = 200, gamma = 2, subsample = 1, colsample_bytree = 0.5, early_stopping_rounds = 250)
model.fit(X_train, y_train, eval_set = ([(X_test, y_test)]))

[0]	validation_0-rmse:8209.83863
[1]	validation_0-rmse:8208.37944
[2]	validation_0-rmse:8206.93519
[3]	validation_0-rmse:8205.46474
[4]	validation_0-rmse:8203.92908
[5]	validation_0-rmse:8202.46552
[6]	validation_0-rmse:8201.02365
[7]	validation_0-rmse:8199.49399
[8]	validation_0-rmse:8197.97636
[9]	validation_0-rmse:8196.47308
[10]	validation_0-rmse:8194.97505
[11]	validation_0-rmse:8193.45698
[12]	validation_0-rmse:8191.96341
[13]	validation_0-rmse:8190.52916
[14]	validation_0-rmse:8189.05478
[15]	validation_0-rmse:8187.54469
[16]	validation_0-rmse:8186.07007
[17]	validation_0-rmse:8184.67133
[18]	validation_0-rmse:8183.25575
[19]	validation_0-rmse:8181.84447
[20]	validation_0-rmse:8180.45119
[21]	validation_0-rmse:8179.05647
[22]	validation_0-rmse:8177.64483
[23]	validation_0-rmse:8176.18328
[24]	validation_0-rmse:8174.72279
[25]	validation_0-rmse:8173.33634
[26]	validation_0-rmse:8171.92569
[27]	validation_0-rmse:8170.48934
[28]	validation_0-rmse:8169.11995
[29]	validation_0-rmse:8

[238]	validation_0-rmse:7917.30050
[239]	validation_0-rmse:7916.28846
[240]	validation_0-rmse:7915.26184
[241]	validation_0-rmse:7914.31906
[242]	validation_0-rmse:7913.26907
[243]	validation_0-rmse:7912.27271
[244]	validation_0-rmse:7911.29034
[245]	validation_0-rmse:7910.31573
[246]	validation_0-rmse:7909.31417
[247]	validation_0-rmse:7908.35466
[248]	validation_0-rmse:7907.35736
[249]	validation_0-rmse:7906.40377
[250]	validation_0-rmse:7905.41079
[251]	validation_0-rmse:7904.42313
[252]	validation_0-rmse:7903.49861
[253]	validation_0-rmse:7902.57001
[254]	validation_0-rmse:7901.57127
[255]	validation_0-rmse:7900.58424
[256]	validation_0-rmse:7899.62842
[257]	validation_0-rmse:7898.70715
[258]	validation_0-rmse:7897.70174
[259]	validation_0-rmse:7896.70329
[260]	validation_0-rmse:7895.75556
[261]	validation_0-rmse:7894.76710
[262]	validation_0-rmse:7893.79594
[263]	validation_0-rmse:7892.83741
[264]	validation_0-rmse:7891.86177
[265]	validation_0-rmse:7890.87646
[266]	validation_0-r

[473]	validation_0-rmse:7723.66070
[474]	validation_0-rmse:7723.04039
[475]	validation_0-rmse:7722.42927
[476]	validation_0-rmse:7721.80637
[477]	validation_0-rmse:7721.18058
[478]	validation_0-rmse:7720.47659
[479]	validation_0-rmse:7719.80982
[480]	validation_0-rmse:7719.18792
[481]	validation_0-rmse:7718.53256
[482]	validation_0-rmse:7717.90666
[483]	validation_0-rmse:7717.22461
[484]	validation_0-rmse:7716.53484
[485]	validation_0-rmse:7715.89591
[486]	validation_0-rmse:7715.23076
[487]	validation_0-rmse:7714.57379
[488]	validation_0-rmse:7713.91980
[489]	validation_0-rmse:7713.25965
[490]	validation_0-rmse:7712.64591
[491]	validation_0-rmse:7712.00308
[492]	validation_0-rmse:7711.37468
[493]	validation_0-rmse:7710.75743
[494]	validation_0-rmse:7710.10321
[495]	validation_0-rmse:7709.48814
[496]	validation_0-rmse:7708.87618
[497]	validation_0-rmse:7708.25692
[498]	validation_0-rmse:7707.61412
[499]	validation_0-rmse:7706.96725
[500]	validation_0-rmse:7706.36494
[501]	validation_0-r

[708]	validation_0-rmse:7595.60427
[709]	validation_0-rmse:7595.16242
[710]	validation_0-rmse:7594.72761
[711]	validation_0-rmse:7594.28293
[712]	validation_0-rmse:7593.83372
[713]	validation_0-rmse:7593.37276
[714]	validation_0-rmse:7592.95482
[715]	validation_0-rmse:7592.53327
[716]	validation_0-rmse:7592.06192
[717]	validation_0-rmse:7591.62762
[718]	validation_0-rmse:7591.15906
[719]	validation_0-rmse:7590.70285
[720]	validation_0-rmse:7590.27852
[721]	validation_0-rmse:7589.84607
[722]	validation_0-rmse:7589.40016
[723]	validation_0-rmse:7588.95471
[724]	validation_0-rmse:7588.53978
[725]	validation_0-rmse:7588.10245
[726]	validation_0-rmse:7587.68004
[727]	validation_0-rmse:7587.24634
[728]	validation_0-rmse:7586.79034
[729]	validation_0-rmse:7586.37606
[730]	validation_0-rmse:7585.91588
[731]	validation_0-rmse:7585.48880
[732]	validation_0-rmse:7585.06038
[733]	validation_0-rmse:7584.62826
[734]	validation_0-rmse:7584.18470
[735]	validation_0-rmse:7583.75470
[736]	validation_0-r

[943]	validation_0-rmse:7509.98416
[944]	validation_0-rmse:7509.70465
[945]	validation_0-rmse:7509.44513
[946]	validation_0-rmse:7509.12024
[947]	validation_0-rmse:7508.83203
[948]	validation_0-rmse:7508.52060
[949]	validation_0-rmse:7508.23254
[950]	validation_0-rmse:7507.93259
[951]	validation_0-rmse:7507.64278
[952]	validation_0-rmse:7507.34183
[953]	validation_0-rmse:7507.03520
[954]	validation_0-rmse:7506.72803
[955]	validation_0-rmse:7506.43027
[956]	validation_0-rmse:7506.15594
[957]	validation_0-rmse:7505.84930
[958]	validation_0-rmse:7505.55553
[959]	validation_0-rmse:7505.23439
[960]	validation_0-rmse:7504.93817
[961]	validation_0-rmse:7504.63737
[962]	validation_0-rmse:7504.35187
[963]	validation_0-rmse:7504.08098
[964]	validation_0-rmse:7503.82739
[965]	validation_0-rmse:7503.55135
[966]	validation_0-rmse:7503.26503
[967]	validation_0-rmse:7503.01601
[968]	validation_0-rmse:7502.72728
[969]	validation_0-rmse:7502.46123
[970]	validation_0-rmse:7502.17219
[971]	validation_0-r

...

[4364]	validation_0-rmse:7336.54366
[4365]	validation_0-rmse:7336.54154
[4366]	validation_0-rmse:7336.54869
[4367]	validation_0-rmse:7336.54812
[4368]	validation_0-rmse:7336.54308
[4369]	validation_0-rmse:7336.53687
[4370]	validation_0-rmse:7336.52966
[4371]	validation_0-rmse:7336.53384
[4372]	validation_0-rmse:7336.53953
[4373]	validation_0-rmse:7336.54819
[4374]	validation_0-rmse:7336.55217
[4375]	validation_0-rmse:7336.55470
[4376]	validation_0-rmse:7336.55204
[4377]	validation_0-rmse:7336.54975
[4378]	validation_0-rmse:7336.54918
[4379]	validation_0-rmse:7336.55445
[4380]	validation_0-rmse:7336.55098
[4381]	validation_0-rmse:7336.55245
[4382]	validation_0-rmse:7336.55975
[4383]	validation_0-rmse:7336.56054
[4384]	validation_0-rmse:7336.57172
[4385]	validation_0-rmse:7336.56440
[4386]	validation_0-rmse:7336.56562
[4387]	validation_0-rmse:7336.56884
[4388]	validation_0-rmse:7336.57828
[4389]	validation_0-rmse:7336.58522
[4390]	validation_0-rmse:7336.57928
[4391]	validation_0-rmse:733

[4592]	validation_0-rmse:7336.46853
[4593]	validation_0-rmse:7336.46601
[4594]	validation_0-rmse:7336.47063
[4595]	validation_0-rmse:7336.46675
[4596]	validation_0-rmse:7336.47269
[4597]	validation_0-rmse:7336.46852
[4598]	validation_0-rmse:7336.46721
[4599]	validation_0-rmse:7336.47191
[4600]	validation_0-rmse:7336.47305
[4601]	validation_0-rmse:7336.47038
[4602]	validation_0-rmse:7336.46418
[4603]	validation_0-rmse:7336.46276
[4604]	validation_0-rmse:7336.44686
[4605]	validation_0-rmse:7336.44509
[4606]	validation_0-rmse:7336.45801
[4607]	validation_0-rmse:7336.46006
[4608]	validation_0-rmse:7336.45842
[4609]	validation_0-rmse:7336.45907
[4610]	validation_0-rmse:7336.46007
[4611]	validation_0-rmse:7336.45950
[4612]	validation_0-rmse:7336.46666
[4613]	validation_0-rmse:7336.46959
[4614]	validation_0-rmse:7336.46374
[4615]	validation_0-rmse:7336.46563
[4616]	validation_0-rmse:7336.46727
[4617]	validation_0-rmse:7336.47011
[4618]	validation_0-rmse:7336.47088
[4619]	validation_0-rmse:733

[4820]	validation_0-rmse:7336.39013
[4821]	validation_0-rmse:7336.39163
[4822]	validation_0-rmse:7336.39633
[4823]	validation_0-rmse:7336.39479
[4824]	validation_0-rmse:7336.38995
[4825]	validation_0-rmse:7336.38809
[4826]	validation_0-rmse:7336.38044
[4827]	validation_0-rmse:7336.38090
[4828]	validation_0-rmse:7336.37832
[4829]	validation_0-rmse:7336.37603
[4830]	validation_0-rmse:7336.37421
[4831]	validation_0-rmse:7336.37396
[4832]	validation_0-rmse:7336.37049
[4833]	validation_0-rmse:7336.37018
[4834]	validation_0-rmse:7336.36991
[4835]	validation_0-rmse:7336.36959
[4836]	validation_0-rmse:7336.36754
[4837]	validation_0-rmse:7336.37601
[4838]	validation_0-rmse:7336.37366
[4839]	validation_0-rmse:7336.37664
[4840]	validation_0-rmse:7336.37231
[4841]	validation_0-rmse:7336.36881
[4842]	validation_0-rmse:7336.36409
[4843]	validation_0-rmse:7336.36834
[4844]	validation_0-rmse:7336.37351
[4845]	validation_0-rmse:7336.37273
[4846]	validation_0-rmse:7336.37031
[4847]	validation_0-rmse:733

[5048]	validation_0-rmse:7336.45211
[5049]	validation_0-rmse:7336.44900
[5050]	validation_0-rmse:7336.44837
[5051]	validation_0-rmse:7336.44205
[5052]	validation_0-rmse:7336.44416
[5053]	validation_0-rmse:7336.44133
[5054]	validation_0-rmse:7336.42015
[5055]	validation_0-rmse:7336.41618
[5056]	validation_0-rmse:7336.41709
[5057]	validation_0-rmse:7336.42306
[5058]	validation_0-rmse:7336.42773
[5059]	validation_0-rmse:7336.42279
[5060]	validation_0-rmse:7336.42464
[5061]	validation_0-rmse:7336.42704
[5062]	validation_0-rmse:7336.44031
[5063]	validation_0-rmse:7336.44576
[5064]	validation_0-rmse:7336.45234
[5065]	validation_0-rmse:7336.45667
[5066]	validation_0-rmse:7336.45486
[5067]	validation_0-rmse:7336.46430
[5068]	validation_0-rmse:7336.45765
[5069]	validation_0-rmse:7336.45195
[5070]	validation_0-rmse:7336.45212
[5071]	validation_0-rmse:7336.44829
[5072]	validation_0-rmse:7336.45104
[5073]	validation_0-rmse:7336.44964
[5074]	validation_0-rmse:7336.45250
[5075]	validation_0-rmse:733

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, early_stopping_rounds=250,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=2, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.001, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=4, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=20000, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=1, ...)

In [28]:
print('XGBoost MAE is {}'.format(mean_absolute_error(y_test, model.predict(X_test))))

XGBoost MAE is 2839.0195985280006
